In [1]:
import pandas as pd

In [2]:
from pybaseball import batting_stats, batting_stats_bref

In [3]:
statcast = pd.read_csv("data/pitches_2016_2018.csv")

In [4]:
statcast = statcast.loc[(statcast["game_date"] < "2019-01-01") & (statcast["game_date"] > "2018-01-01")]

In [5]:
b = batting_stats(2018)

In [6]:
def get_pca5_series_pitcher(df):
    pab = df.groupby(["pitcher", "game_date", "batter", "inning"]).size()
    return pab.loc[pab > 5].to_frame().reset_index().groupby("pitcher").size()

In [7]:
def get_pca5_series_batter(df):
    bab = df.groupby(["batter", "game_date", "inning"]).size()
    return bab.loc[bab > 5].to_frame().reset_index().groupby("batter").size()

In [8]:
def get_bf(df):
    return df.loc[df["events"].notna()].groupby(["pitcher"]).size()

In [9]:
# returns tuple (release_speed_mean, effective_speed_mean, pfx_x_mean, pfx_z_mean)
def get_movement_series(df):
    d = df.groupby("pitcher").agg({"release_speed": "mean",
                                 "effective_speed": "mean",
                                 "pfx_x": "mean", 
                                 "pfx_z": "mean"}).abs().rename(index=str, columns={"release_speed": "release_speed_mean",
                                                                 "effective_speed": "effective_speed_mean",
                                                                 "pfx_x": "pfx_x_mean",
                                                                 "pfx_z": "pfx_z_mean"})
    return (d["release_speed_mean"], d["effective_speed_mean"], d["pfx_x_mean"], d["pfx_z_mean"])

In [10]:
def get_event_series_pitcher(df, event):
    return df.loc[(df["events"] == event)].groupby("pitcher").size()

def get_event_series_batter(df, event):
    return df.loc[(df["events"] == event)].groupby("batter").size()

In [11]:
# this only estimates ip, it is not perfect
def get_ip_series(df):
    return df.groupby(["pitcher", "game_date"])["inning"].nunique().to_frame().reset_index().groupby("pitcher")["inning"].sum()

In [12]:
def get_whip_series(df):
    walks = df.loc[(df["events"] == "walk")].groupby("pitcher").size()
    hits = df.loc[(df["events"] == "single") |
                        (df["events"] == "double") |
                        (df["events"] == "triple") |
                        (df["events"] == "home_run")].groupby("pitcher").size()
    stuff = pd.concat([get_ip_series(df), walks, hits], keys=["ip", "walks", "hits"], axis=1)
    return (stuff["walks"] + stuff["hits"]) / stuff["ip"]

In [13]:
def remove_infreq_pitchers(df):
    bf = get_bf(df)
    bf.loc[bf > 70].index

    return df.loc[(df["pitcher"].isin(bf.loc[bf > 70].index))]

def make_pitcher_df(df):
    df = remove_infreq_pitchers(df)
    _, avg_speed, avg_x, avg_z = get_movement_series(df)
    return pd.concat([get_event_series_pitcher(df, "strikeout"),
                      get_event_series_pitcher(df, "home_run"),
                      get_event_series_pitcher(df, "walk"), # I don't think we can do IBB
                      get_pca5_series_pitcher(df),
                      get_bf(df),
                      get_whip_series(df),
                      avg_speed,
                      avg_x,
                      avg_z
                     ],
                     keys=["SO", "HR", "BB", "PCA5", "BF", "WHIP", "avg_speed", "avg_x", "avg_z"],
                     axis=1)

def make_efp_series(pitcher_df):
    df = pitcher_df
    # WE DON'T HAVE IBB, only BB
    return (1.0*df["SO"] - (0.5*df["PCA5"] + 3.0*df["HR"] + 3.0*df["BB"])) / df["BF"] - \
            1.0*df["WHIP"] + 0.1*df["avg_z"] + 0.1*df["avg_x"] + 0.3*df["avg_speed"]

In [14]:
from pybaseball.playerid_lookup import get_lookup_table
lookup_table = get_lookup_table()

def playerid_lookup_c(last, first=None, year=None):
    # force input strings to lowercase
    last = last.lower()
    if first:
        first = first.lower()
    table = lookup_table
    
    if first is None:
        if year is not None:
            results = table.loc[(table['name_last'] == last) &
                                (table['mlb_played_first'] <= year) &
                                (table['mlb_played_last'] >= year)]
        else:
            results = table.loc[table['name_last'] == last]
    else:
        if year is not None:
            results = table.loc[(table['name_last'] == last) & (table['name_first'] == first) &
                                (table['mlb_played_first'] <= year) &
                                (table['mlb_played_last'] >= year)]
        else:
            results = results = table.loc[(table['name_last'] == last) & (table['name_first'] == first)]

        iterations = 0
        while len(results) == 0 and iterations < 10:
            iterations += 1
            if 'jr.' in last:
                last = last.replace('jr.', '').strip()
            elif 'jr' in last:
                last = last.replace('jr', '').strip()
            elif '.' in first and ' ' in first:
                first = first.replace(' ', '').replace('.', '')
            elif '.' in first:
                first = first[:first.index('.') + 1] + ' ' + first[first.index('.') + 1:]
            elif first == 'nicholas':
                first = 'nick'
            elif first == 'yolmer' and last == 'sanchez':
                first = 'carlos'
            elif first == 'raffy' and last == 'lopez':
                first = 'rafael'
            elif last == 'ervin':
                first = 'phil'
            elif last == 'wheeler' and first == 'zack':
                first = 'zach'
            elif last == 'joyce' and first == 'matt':
                first = 'matthew'
            elif last == 'vogelbach' and first == 'daniel':
                first = 'dan'
            elif last == 'kang':
                first = 'jung ho'
            elif last == 'urshela':
                first = 'gio'
            elif last == 'poncedeleon':
                last = 'ponce de leon'
            elif last == 'bowman' and first == 'matt':
                first = 'matthew'
            elif last == 'chargois':
                first = 'j. t.'
            elif last == 'boyd' and first == 'matthew':
                first = 'matt'
            elif last == 'gosselin' and first == 'phil':
                first = 'philip'
            elif last == 'guerra' and first == 'javier':
                first = 'javy'
            elif last == 'delmonico' and first == 'nicky':
                first = 'nick'
            elif last == 'wilkerson' and first == 'steve':
                first = 'stevie'
            elif first == 'john' and last == 'ryan murphy':
                first = 'j. r.'
                last = 'murphy'
            else:
                for name_bit in last.split(' '):
                    if len(name_bit) <= 3:
                        last = last.replace(name_bit, '').strip()

            if year is not None:
                results = table.loc[(table['name_last'] == last) & (table['name_first'] == first) &
                                    (table['mlb_played_first'] <= year) &
                                    (table['mlb_played_last'] >= year)]
            else:
                results = results = table.loc[(table['name_last'] == last) &
                                              (table['name_first'] == first)]
    
    
    #results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']] = results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']].astype(int) # originally returned as floats which is wrong
    results = results.reset_index().drop('index', 1)
    return results

def get_mlbam_from_name(last, first=None, year=None):
    try:
        return playerid_lookup_c(last, first, year=year).dropna().reset_index(drop=True)["key_mlbam"].iloc[0]
    except IndexError as e:
        if '.' in first and ' ' in first:
            first = first.replace(' ', '').replace('.', '')
        elif '.' in first:
            first = first[:first.index('.') + 1] + ' ' + first[first.index('.') + 1:]
        elif first.lower() == 'nicholas':
            first = 'nick'
        elif first.lower() == 'yolmer' and last.lower() == 'sanchez':
            first = 'carlos'
        else:
            #raise Exception("your name bad: %s, %s" % (last, first))
            print("bad name: %s, %s" % (last, first))
            return -1
        
        return get_mlbam_from_name(last, first)
        return playerid_lookup_c(last, first).dropna().reset_index(drop=True)["key_mlbam"].iloc[0]
        # j.d. martinez -> j. d. martinez

def add_mlbam_to_fg(df_fg):
    def get_last_first(name):
        s = name.split(' ')
        last, first = ' '.join(s[1:]), s[0]
        return last, first
    
    def get_mlbam(name):
        last, first = get_last_first(name)
        return get_mlbam_from_name(last, first)
    
    df_fg["key_mlbam"] = df_fg["Name"].apply(get_mlbam)
    return df_fg

Gathering player lookup table. This may take a moment.


In [15]:
def remove_infreq_batters(df_fangraphs):
    return df_fangraphs.loc[(df_fangraphs["AB"] > 50)]

def make_batter_df(df_fangraphs, df_statcast):
    df_fg = remove_infreq_batters(df_fangraphs).copy()
    df_fg = add_mlbam_to_fg(df_fg)
    pca5 = get_pca5_series_batter(df_statcast).rename("PCA5")
    df_fg = df_fg.merge(pca5.to_frame(), left_on="key_mlbam", right_on="batter")
    df_fg = df_fg.set_index("key_mlbam")
    return df_fg

def make_efb_series(batter_df):
    df = batter_df
    return (((1.0 * df["HR"] + 0.8*df["3B"] + 0.5*df["2B"] + 0.2*df["1B"] + 0.5*df["SB"] - \
           (0.4*df["SO"] + 0.2*df["BB"] + 0.2*df["PCA5"])) / df["PA"]) + 2) * 10

In [16]:
efb_s = make_efb_series(make_batter_df(b, statcast)).sort_values(ascending=False)

In [17]:
b_df = make_batter_df(b, statcast)

In [22]:
efb_s.rename("EFB").to_frame().merge(b_df[["Name", "Team", "HR", "3B", "2B", "1B", "SB", "SO", "BB", "PCA5", "PA"]], left_index=True, right_index=True).to_json(orient="columns")

'{"EFB":{"553902":20.6804123711,"643446":20.3709677419,"606299":20.2723279649,"592743":20.27,"516416":20.2262658228,"645302":20.2121212121,"488726":20.206022187,"578428":20.1939655172,"501571":20.15625,"609275":20.147766323,"609280":20.1435643564,"493329":20.1431064572,"543829":20.130952381,"605141":20.1221498371,"544369":20.1072056239,"572816":20.0844277674,"649557":20.064159292,"596019":20.0604026846,"435062":20.05625,"435559":20.0515463918,"502517":20.0455840456,"516782":20.0445544554,"592518":20.0211565585,"593160":20.0056577086,"596748":20.0,"405395":19.9979919679,"592696":19.9949324324,"527038":19.9836829837,"456488":19.9800796813,"607680":19.9778597786,"430945":19.9738988581,"622569":19.9682539683,"645277":19.9561403509,"514888":19.9465776294,"542255":19.9439393939,"592885":19.9324116743,"607345":19.9304812834,"595879":19.9302325581,"502082":19.9263157895,"519203":19.9203007519,"543685":19.9195979899,"429664":19.9137931034,"605412":19.9132653061,"466320":19.8992805755,"664058":1

In [20]:
efp_s = efp_s.rename("EFP")

In [23]:
from pybaseball import playerid_lookup, playerid_reverse_lookup

In [24]:
ids = []
for i in efp_s.index:
    ids.append(i)

pitcher_ids = playerid_reverse_lookup(ids)

Gathering player lookup table. This may take a moment.


In [ ]:
#     return (1.0*df["SO"] - (0.5*df["PCA5"] + 3.0*df["HR"] + 3.0*df["BB"])) / df["BF"] - \
#             1.0*df["WHIP"] + 0.1*df["avg_z"] + 0.1*df["avg_x"] + 0.3*df["avg_speed"]

In [32]:
p_df = make_pitcher_df(statcast)
efp_s = make_efp_series(p_df).rename("EFP")
combined = p_df.merge(efp_s.to_frame(), left_index=True, right_index=True)
with_ids = combined.merge(pitcher_ids[["name_last", "name_first", "key_mlbam"]], left_index=True, right_on="key_mlbam").set_index("key_mlbam", drop=True)
with_ids.to_json(orient="columns")

'{"SO":{"112526":81,"279571":25,"282332":140,"407822":48,"407845":70,"424144":43,"425492":62,"425794":39,"425844":198,"429719":65,"429722":16,"430641":9,"430935":189,"433586":22,"433587":124,"433589":76,"434378":307,"434538":109,"434628":90,"434671":138,"435043":51,"435221":65,"444468":67,"445197":12,"445213":43,"445276":95,"445926":92,"446099":53,"446372":222,"446399":19,"446899":50,"448178":8,"448179":166,"448281":60,"448306":151,"448609":44,"448802":72,"448855":140,"450203":202,"450212":15,"450306":84,"450729":40,"451584":81,"451596":56,"451661":33,"452657":157,"453172":58,"453178":105,"453192":45,"453214":42,"453265":72,"453281":128,"453284":70,"453286":299,"453329":81,"453343":30,"453344":31,"453385":108,"453562":137,"455119":37,"456034":200,"456501":38,"456696":65,"456701":75,"456713":19,"457915":30,"457918":195,"458006":32,"458584":35,"458677":69,"458681":161,"458690":29,"458708":46,"458924":28,"459429":47,"460283":41,"461325":84,"461829":147,"461833":32,"461872":30,"462136":102

In [ ]:
playerid_reverse_lookup([553902], key_type="mlbam")